# Assignment

Implementare uno script di classificazione basato sul fine tuning di feature neurali.

Verificare l'impatto sulle performance dato da:
- Una diversa architettura
- Operazioni di data augmentation

# Caricamento librerie e dati

## Import librerie

In [1]:
# Google Colab tensorflow_version
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!pip install -q keras==2.3.0

In [3]:
import keras
print(keras.__version__)

Using TensorFlow backend.


2.3.0


In [0]:
from keras.applications import resnet_v2
from keras.applications import mobilenet_v2
from keras.preprocessing import image as kimage
from keras.models import Model, load_model
from keras.layers import Dense
from keras import optimizers

In [0]:
import numpy as np
from time import time

## Import dati

In [6]:
# Download del dataset
!wget https://www.dropbox.com/s/drwy7fq5svwv78p/101_ObjectCategories_split.tar

--2020-02-11 10:37:32--  https://www.dropbox.com/s/drwy7fq5svwv78p/101_ObjectCategories_split.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/drwy7fq5svwv78p/101_ObjectCategories_split.tar [following]
--2020-02-11 10:37:32--  https://www.dropbox.com/s/raw/drwy7fq5svwv78p/101_ObjectCategories_split.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaad89f4e3dca254243c9830739.dl.dropboxusercontent.com/cd/0/inline/Ax665wKFPkE2FD6Vlr0rqfbRlDK6n8o5NzOMUmebRB_yNFVXnpRuvh0oVdtmf-n_G_95Qb9TiNgmEC6QQ31Bt0Xt9JCXRVYi0CY79mYKbJcF7g/file# [following]
--2020-02-11 10:37:32--  https://ucaad89f4e3dca254243c9830739.dl.dropboxusercontent.com/cd/0/inline/Ax665wKFPkE2FD6Vlr0rqfbRlDK6n8o5NzOMUmebRB_yNFVXnpRuvh0oVdtmf-n_G_95Qb9TiNgmEC6QQ

In [0]:
# Estrazione file e rimozione archivio
import tarfile
tar = tarfile.open('101_ObjectCategories_split.tar')
tar.extractall()
tar.close()
!rm 101_ObjectCategories_split.tar

In [0]:
base_path = '101_ObjectCategories_split/'

# Fine-tuning con Mobilenet V2

## Data loader

### Training set

In [9]:
# Applicazione del preprocessing 'standard' per mobilenet su train_set
train_processing = kimage.ImageDataGenerator(
    preprocessing_function=mobilenet_v2.preprocess_input)

# creazione train_generator
train_generator = train_processing.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)

Found 4600 images belonging to 102 classes.


### Test set

In [10]:
# Applicazione del preprocessing 'standard' per mobilenet su test set
test_processing = kimage.ImageDataGenerator(
    preprocessing_function=mobilenet_v2.preprocess_input)

# creazione test_generator
test_generator = test_processing.flow_from_directory(
        directory=base_path+'test',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 4544 images belonging to 102 classes.


## Caricamento modello di base

In questa fase viene caricato il modello mobilnet pre-addestrato sul dataset 'imagenet'. Viene caricata solo la parte convoluzionale del modello. Il classificatore fully-connected non è incluso.

In [0]:
# Modello di base
mob_net = mobilenet_v2.MobileNetV2(input_shape=(224,224,3),
                                   weights='imagenet',
                                   include_top=False,
                                   pooling='avg')

## Creazione del classificatore

Vengono congelati i pesi della rete. Così facendo in fase di addestramento verranno addestrati solo i pesi del classificatore fully-connected. I pesi della mobilenet rimarranno invariati.


La rete potrebbe anche essere 'tagliata' prima in modo tale che le features estratte siano più generiche. Ma, essendo il dataset utilizzato abbastanza simile ad imagenet questa operazione non è stata effettuata.

In [0]:
# Congelamento layers del modello di base (fine-tuning)
for layer in mob_net.layers:
    layer.trainable = False

Viene aggiunto il classificatore fully-connected costituito da:
- Fully-connected che mappa a 1024 dimensioni
- ReLU
- Fully-connected che mappi alla dimensione finale (102 classi)

In [0]:
# Output del modello di base
x_mob = mob_net.output

# Classificatore fully-connected
x_mob = Dense(1024, activation='relu')(x_mob)
pred_mob = Dense(102, activation='softmax')(x_mob)

Vengono unite la mobilnet ed il classificatore

In [0]:
# Modello specializzato
my_mob_net = Model(inputs=mob_net.input, outputs=pred_mob)

## Addestramento (fine-tuning)

In [0]:
# Compila il modello per l'addestramento
my_mob_net.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizers.RMSprop(),
                  metrics=['accuracy'])

In [16]:
my_mob_net.fit_generator(train_generator,
                         epochs=3,
                         validation_data=test_generator)

Epoch 1/3
144/144 [==============================] - 30s 207ms/step - loss: 1.5962 - accuracy: 0.6552 - val_loss: 0.5109 - val_accuracy: 0.8083
Epoch 2/3
144/144 [==============================] - 24s 165ms/step - loss: 0.4351 - accuracy: 0.8776 - val_loss: 0.6177 - val_accuracy: 0.8426
Epoch 3/3
144/144 [==============================] - 24s 165ms/step - loss: 0.2190 - accuracy: 0.9289 - val_loss: 0.2435 - val_accuracy: 0.8649


## Valutazione del modello

In [17]:
my_mob_net.evaluate_generator(test_generator)

[0.24346624314785004, 0.8648767471313477]

Il modello è stato testato utilizzando il test generator ed è possibile osservare che raggiunge l'86% di accuracy.

## Data-Augmentation

### Training set

Creazione di un nuovo train generator in cui, oltre al preprocessing standard della mobilmet vegono applicati anche diversi metodi di data augmentation, tra cui:
- width_shift_range: l'immagine viene 'spostata' in orizzontale
- height_shift_range: l'immagine viene 'spostata' in vertivale
- horizontal_flip: flip dell'immagine in orizzontale
- vertical_flip: flip dell'immagine in verticale
- rotation_range: l'immagine viene ruotata
- brightness_range: viene modificata la brightness dell'immagine


In [18]:
train_processing = kimage.ImageDataGenerator(
    preprocessing_function = mobilenet_v2.preprocess_input,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 90,
    brightness_range = [0.2,1.0]
    )

# creazione train_generator
train_generator = train_processing.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)

Found 4600 images belonging to 102 classes.


### Addestramento modello

Viene utilizzato lo stesso modello creato precedentemente ma con il train generator in cui è stata utilizzata la data augmentation

In [0]:
# Modello specializzato
my_mob_net = Model(inputs=mob_net.input, outputs=pred_mob)

# Compila il modello per l'addestramento
my_mob_net.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizers.RMSprop(),
                  metrics=['accuracy'])

In [20]:
my_mob_net.fit_generator(
    train_generator, epochs=3, validation_data=test_generator
    )

Epoch 1/3
144/144 [==============================] - 81s 565ms/step - loss: 1.6044 - accuracy: 0.6128 - val_loss: 0.7346 - val_accuracy: 0.8024
Epoch 2/3
144/144 [==============================] - 74s 514ms/step - loss: 1.2093 - accuracy: 0.6839 - val_loss: 0.6318 - val_accuracy: 0.8202
Epoch 3/3
144/144 [==============================] - 75s 518ms/step - loss: 1.0656 - accuracy: 0.7170 - val_loss: 0.7882 - val_accuracy: 0.8233


### Valutazione del modello

In [21]:
my_mob_net.evaluate_generator(test_generator)

[0.47388771176338196, 0.8232834339141846]

Con l'augmentation dei dati l'accuracy è pari all'82%, dunque più bassa rispetto al modello senza augmentation. È da considerare, però, che il modello è stato addestrato con sole tre iterazioni.

# Fine-tuning con ResNet

Le stesse operazioni eseguite con la rete mobilnet vengono ripetute con la rete ResNet per confrontarne i risultati.

## Data loader



### Training set

In [22]:
# Applicazione del preprocessing 'standard' per resnet su train_set
train_processing_res = kimage.ImageDataGenerator(
    preprocessing_function = resnet_v2.preprocess_input
    )

# creazione train_generator
train_generator_res = train_processing_res.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)

Found 4600 images belonging to 102 classes.


### Test set

In [23]:
# Applicazione del preprocessing 'standard' per resnet su test set
test_processing_res = kimage.ImageDataGenerator(
    preprocessing_function = resnet_v2.preprocess_input
    )

# creazione test_generator
test_generator_res = test_processing_res.flow_from_directory(
        directory=base_path+'test',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 4544 images belonging to 102 classes.


## Caricamento modello di base

In [0]:
# Modello di base
resnet = resnet_v2.ResNet101V2(
    input_shape=(224,224,3),
    weights='imagenet',
    include_top=False,
    pooling='avg'
    )

## Creazione del classificatore

In [0]:
# Congelamento layers del modello di base (fine-tuning)
for layer in resnet.layers:
    layer.trainable = False

In [0]:
# Output del modello di base
x_res = resnet.output

# Classificatore fully-connected
x_res = Dense(1024, activation='relu')(x_res)
pred_res = Dense(102, activation='softmax')(x_res)

In [0]:
# Modello specializzato
my_resnet = Model(inputs=resnet.input, outputs=pred_res)

## Addestramento (fine-tuning)

In [0]:
# Compila il modello per l'addestramento
my_resnet.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizers.RMSprop(),
                  metrics=['accuracy'])

In [29]:
my_resnet.fit_generator(
    train_generator_res, epochs=3, validation_data=test_generator_res
    )

Epoch 1/3
144/144 [==============================] - 73s 505ms/step - loss: 1.1041 - accuracy: 0.7415 - val_loss: 0.7102 - val_accuracy: 0.8352
Epoch 2/3
144/144 [==============================] - 64s 445ms/step - loss: 0.3005 - accuracy: 0.9089 - val_loss: 0.7590 - val_accuracy: 0.8939
Epoch 3/3
144/144 [==============================] - 64s 443ms/step - loss: 0.1614 - accuracy: 0.9574 - val_loss: 0.2766 - val_accuracy: 0.8935


In [30]:
my_resnet.evaluate_generator(test_generator_res)

[0.5751411318778992, 0.8934859037399292]

La resnet utilizzando solo tre iterazioni sembra ottenere risultati migliori rispetto alla mobilnet. L'accuracy, infatti, è pari all'89%.

## Data-Augmentation

### Training set

In [31]:
train_processing_res = kimage.ImageDataGenerator(
    preprocessing_function=resnet_v2.preprocess_input,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    brightness_range=[0.2,1.0]
    )

# creazione train_generator
train_generator_res = train_processing_res.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)

Found 4600 images belonging to 102 classes.


### Addestramento modello

Viene utilizzato lo stesso modello creato precedentemente ma con il train generator in cui è stata utilizzata la data augmentation

In [0]:
# Modello specializzato
my_resnet = Model(inputs=resnet.input, outputs=pred_res)

# Compila il modello per l'addestramento
my_resnet.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizers.RMSprop(),
                  metrics=['accuracy'])

In [33]:
my_resnet.fit_generator(
    train_generator_res, epochs=3, validation_data=test_generator_res
    )

Epoch 1/3
144/144 [==============================] - 109s 755ms/step - loss: 1.2456 - accuracy: 0.6915 - val_loss: 0.7792 - val_accuracy: 0.8704
Epoch 2/3
144/144 [==============================] - 98s 684ms/step - loss: 0.9034 - accuracy: 0.7557 - val_loss: 0.6582 - val_accuracy: 0.8783
Epoch 3/3
144/144 [==============================] - 99s 685ms/step - loss: 0.8006 - accuracy: 0.7750 - val_loss: 0.3955 - val_accuracy: 0.8772


### Valutazione del modello

In [34]:
my_resnet.evaluate_generator(test_generator_res)

[1.2365059852600098, 0.877200722694397]

La resnet con la data augmentatio risulta pegiorare leggermente, passando dall'89% all'88% di accuracy. Anche in questo caso resnet risulta essere migliore di mobilenet.